In [1]:
import ee

In [5]:
ee.Authenticate()

True

In [89]:
ee.Initialize(project="co2calculator-407613")

In [134]:
#Load country data
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Malaysia = countries.filter(ee.Filter.eq('country_na', 'Malaysia'))

In [135]:
dataset = ee.ImageCollection("NASA/ORNL/biomass_carbon_density/v1")

In [136]:
visualization = {
  'bands': ['agb'],
  'min': -50.0,
  'max': 80.0,
  'palette': ['d9f0a3', 'addd8e', '78c679', '41ab5d', '238443', '005a32']
}

In [137]:
import folium

In [138]:
# Create a folium map object.
my_map = folium.Map(location=[4.2105, 101.9758], zoom_start = 6)

In [139]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [141]:
# Define a function to clip each image in the ImageCollection
def clip_image(image):
    return image.clip(Malaysia)

# Use the map function to apply the clipping function to each image in the collection
clipped_dataset = dataset.map(clip_image)

In [143]:
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Select a specific image from the ImageCollection (e.g., the first one)
selected_image = dataset.first()

# Add the layer to the map object.
my_map.add_ee_layer(selected_image.clip(Malaysia), visualization, 'Malaysia Biomass')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)